In [ ]:
import os, csv
import numpy as np
from tqdm import tqdm
from tensorboard_utils import *
from matplotlib import pyplot as plt
from copy import deepcopy

In [ ]:
LOG_DIR = '/home/lhoyer/cnn_robot_localization/training_results/second_stage_2'
OUT_DIR = '/home/lukas/Nextcloud/Studium/Bachelorarbeit/Plots'

plt.style.use('default')
plt.rcParams.update({'font.size': 14})

exp_dict = {
    'S0': 'sstage_default',
    'S1.1': 'sstage_background_swarmlab',
    'S1.2': 'sstage_background_coco',
    'S2.1': 'sstage_amount_1000',
    'S2.2': 'sstage_amount_20000',
    'S3.1': 'sstage_crop_method_automatic',
    'S3.2': 'sstage_crop_method_both',
    'S4.1': 'sstage_alpha_25',
    'S4.2': 'sstage_alpha_75',
    'S4.3': 'sstage_alpha_100',
}
exp_dict = OrderedDict(sorted(exp_dict.items()))

working_keys = list(exp_dict.keys())
for r in ['copter','sphero','youbot']:
    for m in ['cat','bin']:
        ys = []
        labels = deepcopy(working_keys)
        for k in working_keys:
            recfiles = []
            for root, subdirs, files in os.walk(LOG_DIR):
                if '{}_{}_{}'.format(exp_dict[k],r,m) not in root: continue
                for file in files:
                    recfiles.append(root+'/'+file)
            recfiles.sort()
            #print('{} has {} files: {}'.format(k,len(recfiles),recfiles))

            # Read tfevents
            tensorboards = []
            for file in recfiles:
                if 'tfevents' not in file:
                    continue
                try:
                    tensorboards.append(readTfEvents(file))
                except:
                    print('Couldn\'t read {}'.format(file))


            # Concat tf events
            ys.append([])
            for i,t in enumerate(tensorboards):
                if m == 'cat': filter_range = range(15,25)
                if m == 'bin': filter_range = range(5,15)
                t = filterRange(t, filter_range)
                try:
                    if m == 'cat': 
                        y = (1 - np.array(t['Keras Validation//val_cat_out_acc']))*100
                    if m == 'bin': 
                        y = t['Keras Validation//val_bin_out_angle_bin_error']
                    ys[-1].extend(y)
                except KeyError as e:
                    print('Couldn\' access tensorboard {}: {}'.format(i,e))
                except TypeError as e:
                    print('Couldn\' read tensorboard {}: {}'.format(i,e))
               
        # Add regression
        k = 'S0'
        if m == 'bin':
            labels.append('S5')
            recfiles = []
            for root, subdirs, files in os.walk(LOG_DIR):
                if '{}_{}_{}'.format(exp_dict[k],r,'reg') not in root: continue
                for file in files:
                    recfiles.append(root+'/'+file)
            recfiles.sort()
            # Read tfevents
            tensorboards = []
            for file in recfiles:
                if 'tfevents' not in file:
                    continue
                try:
                    tensorboards.append(readTfEvents(file))
                except:
                    print('Couldn\'t read {}'.format(file))
            # Concat tf events
            ys.append([])
            for i,t in enumerate(tensorboards):
                filter_range = range(10,20)
                t = filterRange(t, filter_range)
                try:
                    y = t['Keras Validation//val_reg_out_angle_mae']
                    ys[-1].extend(y)
                except KeyError as e:
                    print('Couldn\' access tensorboard {}: {}'.format(i,e))
                except TypeError as e:
                    print('Couldn\' read tensorboard {}: {}'.format(i,e))

        # Draw boxplots
        print('Number of default samples: {}'.format([len(l) for l in ys]))
        bp_dict = plt.boxplot(ys,
                    labels=labels,
                    sym='k+',
                    medianprops=dict(color='red'),
                    widths=0.3,
                    showfliers=False
                   )
        for line in bp_dict['medians']:
            # get position data for median line
            x, y = line.get_xydata()[1] # top of median line
            # overlay median value
            plt.text(x+0.05, y, ('%.1f'%y).lstrip('0') if m=='cat' else '%.1f'%y,
                 horizontalalignment='left',
                 verticalalignment='center',
                 fontsize=8) # draw above, centered
            
        plt.ylabel('(1 - Accuracy) in %' if m=='cat' else 'Mean Absolute Angle Difference in °')
        plt.xlabel('Experiment', labelpad=15)
        plt.xticks(rotation=90)
        plt.grid()
        plt.gca().set_xlim([0.5,len(labels)+0.8])
        #if m=='bin' and r=='copter': plt.gca().set_ylim([0,25])
        #if m=='bin' and r=='sphero': plt.gca().set_ylim([5,50])
        #if m=='bin' and r=='youbot': plt.gca().set_ylim([0,40])
        
        #plt.title('Second Stage {} for {}'.format(
        #    'Orientation Estimation' if m=='bin' else 'Identification',
        #    r.capitalize()))
        ax = plt.gca()
        ratio_default=(ax.get_xlim()[1]-ax.get_xlim()[0])/(ax.get_ylim()[1]-ax.get_ylim()[0])
        plt.gca().set_aspect(0.65*ratio_default)
        plt.savefig('{}/second_stage_{}_{}.pdf'.format(OUT_DIR, r,m), 
                    bbox_inches='tight'
                   )
        plt.show()

In [ ]:
"""               
            # Add regression
            if k == 'S0':
                labels.append('S5')
                recfiles = []
                for root, subdirs, files in os.walk(LOG_DIR):
                    if '{}_{}_{}'.format(exp_dict[k],r,'reg') not in root: continue
                    for file in files:
                        recfiles.append(root+'/'+file)
                recfiles.sort()
                # Read tfevents
                tensorboards = []
                for file in recfiles:
                    if 'tfevents' not in file:
                        continue
                    try:
                        tensorboards.append(readTfEvents(file))
                    except:
                        print('Couldn\'t read {}'.format(file))
                # Concat tf events
                ys.append([])
                for i,t in enumerate(tensorboards):
                    filter_range = range(10,20)
                    t = filterRange(t, filter_range)
                    try:
                        y = t['Keras Validation//val_reg_out_angle_mae']
                        ys[-1].extend(y)
                    except KeyError as e:
                        print('Couldn\' access tensorboard {}: {}'.format(i,e))
                    except TypeError as e:
                        print('Couldn\' read tensorboard {}: {}'.format(i,e))
"""

In [ ]:
exp = 'sstage_default'
ys = []
for r in ['copter','sphero','youbot']:
    for m in ['bin','reg']:
        recfiles = []
        for root, subdirs, files in os.walk(LOG_DIR):
            if '{}_{}_{}'.format(exp,r,m) not in root: continue
            for file in files:
                recfiles.append(root+'/'+file)
        recfiles.sort()
        #print('{} has {} files: {}'.format(k,len(recfiles),recfiles))

        # Read tfevents
        tensorboards = []
        for file in recfiles:
            if 'tfevents' not in file:
                continue
            try:
                tensorboards.append(readTfEvents(file))
            except:
                print('Couldn\'t read {}'.format(file))


        # Concat tf events
        ys.append([])
        for i,t in enumerate(tensorboards):
            if m == 'reg': filter_range = range(10,20)
            if m == 'bin': filter_range = range(5,15)
            t = filterRange(t, filter_range)
            try:
                if m == 'reg': 
                    y = np.array(t['Keras Validation//val_reg_out_angle_mae'])
                if m == 'bin': 
                    y = t['Keras Validation//val_bin_out_angle_bin_error']
                ys[-1].extend(y)
            except KeyError as e:
                print('Couldn\' access tensorboard {}: {}'.format(i,e))

bp_dict = plt.boxplot(ys,
            labels=['Copter/C','Copter/R','Sphero/C','Sphero/R','Youbot/C','Youbot/R'],
            sym='k+',
            medianprops=dict(color='red'),
            widths=0.4
            #showfliers=False
           )
for line in bp_dict['medians']:
    # get position data for median line
    x, y = line.get_xydata()[1] # top of median line
    # overlay median value
    plt.text(x+0.05, y, ('%.2f'%y).lstrip('0') if m=='cat' else '%.1f'%y,
         horizontalalignment='left',
         verticalalignment='center',
         fontsize=8) # draw above, centered

plt.ylabel('Mean Absolute Angle Difference in °')
plt.xlabel('Robot type / Estimation Method (C=Classifcation, R=Regression)')
plt.legend(loc='lower right')
plt.grid()
plt.gca().set_xlim([0.5,6.8])
#plt.title('Comparison of Discrete and Continuous Orientation Estimation')
plt.savefig('{}/second_stage_orientation_method.png'.format(OUT_DIR), bbox_inches='tight')
plt.show()

## Presentation Plots

In [ ]:
LOG_DIR = '/home/lhoyer/cnn_robot_localization/training_results/second_stage_2'
OUT_DIR = '/home/lukas/Nextcloud/Studium/Bachelorarbeit/IEEE SSCI Presentation/Plots'

plt.style.use('default')

exp_dict = {
    #'400': 'sstage_amount_1000',
    #'2000': 'sstage_default',
    #'8000': 'sstage_amount_20000',
    '0.25': 'sstage_alpha_25',
    '0.5': 'sstage_default',
    '0.75': 'sstage_alpha_75',
    '1.0': 'sstage_alpha_100',
}
exp_dict = OrderedDict(sorted(exp_dict.items()))

working_keys = list(exp_dict.keys())
for r in ['copter','sphero','youbot']:
    for m in ['cat','bin']:
        ys = []
        labels = deepcopy(working_keys)
        for k in working_keys:
            recfiles = []
            for root, subdirs, files in os.walk(LOG_DIR):
                if '{}_{}_{}'.format(exp_dict[k],r,m) not in root: continue
                for file in files:
                    recfiles.append(root+'/'+file)
            recfiles.sort()
            #print('{} has {} files: {}'.format(k,len(recfiles),recfiles))

            # Read tfevents
            tensorboards = []
            for file in recfiles:
                if 'tfevents' not in file:
                    continue
                try:
                    tensorboards.append(readTfEvents(file))
                except:
                    print('Couldn\'t read {}'.format(file))


            # Concat tf events
            ys.append([])
            for i,t in enumerate(tensorboards):
                if m == 'cat': filter_range = range(15,25)
                if m == 'bin': filter_range = range(5,15)
                t = filterRange(t, filter_range)
                try:
                    if m == 'cat': 
                        y = (1 - np.array(t['Keras Validation//val_cat_out_acc']))*100
                    if m == 'bin': 
                        y = t['Keras Validation//val_bin_out_angle_bin_error']
                    ys[-1].extend(y)
                except KeyError as e:
                    print('Couldn\' access tensorboard {}: {}'.format(i,e))
                except TypeError as e:
                    print('Couldn\' read tensorboard {}: {}'.format(i,e))

        # Draw boxplots
        print('Number of default samples: {}'.format([len(l) for l in ys]))
        bp_dict = plt.boxplot(ys,
                    labels=labels,
                    sym='k+',
                    medianprops=dict(color='red'),
                    widths=0.3,
                    showfliers=False
                   )
        for line in bp_dict['medians']:
            # get position data for median line
            x, y = line.get_xydata()[1] # top of median line
            # overlay median value
            plt.text(x+0.05, y, ('%.1f'%y).lstrip('0') if m=='cat' else '%.1f'%y,
                 horizontalalignment='left',
                 verticalalignment='center',
                 fontsize=9) # draw above, centered
            
        plt.ylabel('(1 - Accuracy) in %' if m=='cat' else 'Mean Absolute Angle Difference in °')
        plt.xlabel('Mobilenet Width Multiplier')
        plt.grid()
        plt.title('{}'.format(r))
        plt.gca().set_xlim([0.5,len(labels)+0.8])
        
        #plt.title('Second Stage {} for {}'.format(
        #    'Orientation Estimation' if m=='bin' else 'Identification',
        #    r.capitalize()))
        ax = plt.gca()
        ratio_default=(ax.get_xlim()[1]-ax.get_xlim()[0])/(ax.get_ylim()[1]-ax.get_ylim()[0])
        plt.gca().set_aspect(1.2*ratio_default)
        plt.savefig('{}/second_stage_width_{}_{}.png'.format(OUT_DIR, r,m), 
                    bbox_inches='tight'
                   )
        plt.show()